# DeepQL with tensorflow

https://github.com/AdrianUng/keras-triplet-loss-mnist/blob/master/Triplet_loss_KERAS_semi_hard_from_TF.ipynb

In [2]:
import keras

In [2]:
from __future__ import print_function, division

In [3]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
from annoy import AnnoyIndex
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [4]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, \
    GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras import optimizers

In [5]:
from methods.baseline import Baseline
from methods.experiments import Experiment
from methods.evaluation import Evaluation
from methods.retrieval import Retrieval

## Configurações Globais

In [6]:
MAX_SEQUENCE_LENGTH_T = 20 # 100
MAX_SEQUENCE_LENGTH_D = 20 # 500
EMBEDDING_DIM = 300
MAX_NB_WORDS = 20000
'''
    Configuration
'''
epochs = 1000
freeze_train = .1
best_loss = 1
best_epoch = 0
verbose = 0
loss = 1

### Parse bugs preproprecessed

In [7]:
# Domain to use
DOMAIN = 'eclipse'
METHOD = 'deepTL_{}'.format(epochs)
PREPROCESSING = 'bert'
TOKEN = 'bert'
# Dataset paths
DIR = 'data/processed/{}/{}'.format(DOMAIN, PREPROCESSING)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Glove embeddings
GLOVE_DIR='data/embed'
# Save model
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

# Extract CORPUs
EXTRACT_CORPUS = False

In [8]:
import os

pretrained_path = 'uncased_L-12_H-768_A-12'
config_path = os.path.join(pretrained_path, 'bert_config.json')
model_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

In [9]:
from keras_bert import load_vocabulary

token_dict = load_vocabulary(vocab_path)

In [10]:
baseline = Baseline(DOMAIN, DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D,
                   token_dict['[CLS]'], token_dict['[SEP]'])
evaluation = Evaluation(verbose=0)
retrieval = Retrieval()
experiment = Experiment(baseline, evaluation)

In [11]:
experiment.set_retrieval(retrieval, baseline, DOMAIN)

#### Loading bug ids in memory

In [12]:
experiment.load_ids()
len(baseline.bug_ids)

Reading bug ids


72234

#### Dicionário de títulos e descrições

In [13]:
%%time

experiment.load_bugs(TOKEN)
len(baseline.sentence_dict)


CPU times: user 6.74 s, sys: 629 ms, total: 7.36 s
Wall time: 7.35 s


#### Hashing bugs by buckets

In [14]:
issues_by_buckets = experiment.get_buckets_for_bugs()

#### Prepare the train and test

In [15]:
# path_train='train_chronological', path_test='test_chronological'
experiment.prepare_dataset(issues_by_buckets, path_train='train_chronological', path_test='test_chronological')
# Read and create the test queries duplicates
retrieval.create_queries()

In [16]:
baseline.train_data[:10]

[[28170, 23787],
 [12823, 22223],
 [23765, 21850],
 [102722, 89006],
 [43930, 46308],
 [32405, 14334],
 [39496, 24193],
 [15393, 17482],
 [8323, 6808],
 [36773, 55415]]

#### Recovery bug ids from train

In [17]:
bug_train_ids = experiment.get_train_ids(baseline.train_data)

### Export the corpus train

In [18]:
if EXTRACT_CORPUS:
    corpus = []
    export_file = open(os.path.join(DIR, 'corpus_train.txt'), 'w')
    for bug_id in tqdm(baseline.bug_set):
        bug = baseline.bug_set[bug_id]
        title = bug['title']
        desc = bug['description']
        export_file.write("{}\n{}\n".format(title, desc))
    export_file.close()

# Generating tiple of batches

In [19]:
idx = np.random.choice(baseline.bug_ids, 1)[0]
baseline.bug_set[idx]

{'bug_severity': '4\n',
 'bug_status': '2\n',
 'component': '8\n',
 'creation_ts': '2006-03-06 09:55:00 +0000',
 'delta_ts': '2013-08-07 14:40:21 +0000',
 'description': '[CLS] load bug doc . there are examples for situations when the promotion / demo ##tion of chapters does not work or does not work correctly . [SEP]',
 'description_segment': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,


### Generating the batch test

In [20]:
"Train ", len(baseline.dup_sets_train)

('Train ', 13626)

In [21]:
import random

# data - path
# batch_size - 128
# n_neg - 1
def batch_iterator(self, retrieval, model, data, dup_sets, bug_ids, 
                   batch_size, n_neg, issues_by_buckets, TRIPLET_HARD=False, FLOATING_PADDING=False):
    # global train_data
    # global self.dup_sets
    # global self.bug_ids
    # global self.bug_set

    random.shuffle(data)

    batch_features = {'title' : [], 'desc' : [], 'info' : []}
    n_train = len(data)

    batch_triplets, batch_bugs_anchor, batch_bugs_pos, batch_bugs_neg, batch_bugs = [], [], [], [], []

    all_bugs = list(issues_by_buckets.keys())
    buckets = retrieval.buckets

    for offset in range(batch_size):
        anchor, pos = data[offset][0], data[offset][1]
        batch_bugs_anchor.append(anchor)
        batch_bugs_pos.append(pos)
        batch_bugs.append(anchor)
        batch_bugs.append(pos)
        #batch_bugs += dup_sets[anchor]

    for anchor, pos in zip(batch_bugs_anchor, batch_bugs_pos):
        while True:
            neg = self.get_neg_bug(anchor, buckets[issues_by_buckets[anchor]], issues_by_buckets, all_bugs)
            bug_anchor = self.bug_set[anchor]
            bug_pos = self.bug_set[pos]
            if neg not in self.bug_set:
                continue
            batch_bugs.append(neg)
            batch_bugs_neg.append(neg)
            bug_neg = self.bug_set[neg]
            break
        
        # triplet bug and master
        batch_triplets.append([anchor, pos, neg])
    
    random.shuffle(batch_bugs)
    title_ids = np.full((len(batch_bugs), MAX_SEQUENCE_LENGTH_T), 0)
    description_ids = np.full((len(batch_bugs), MAX_SEQUENCE_LENGTH_D), 0)
    for i, bug_id in enumerate(batch_bugs):
        bug = self.bug_set[bug_id]
        self.read_batch_bugs(batch_features, bug, index=i, title_ids=title_ids, description_ids=description_ids)

    batch_features['title'] = { 'token' : np.array(batch_features['title']), 'segment' : title_ids }
    batch_features['desc'] = { 'token' : np.array(batch_features['desc']), 'segment' : description_ids }
    batch_features['info'] = np.array(batch_features['info'])
    
    sim = np.asarray([issues_by_buckets[bug_id] for bug_id in batch_bugs])

    input_sample = {}

    input_sample = { 'title' : batch_features['title'], 
                        'description' : batch_features['desc'], 
                            'info' : batch_features['info'] }

    return batch_triplets, input_sample, sim #sim

In [22]:
# %%time

batch_size = 64
batch_size_test = 128

# we want a constant validation group to have a frame of reference for model performance
batch_triplets_valid, valid_input_sample, valid_sim = batch_iterator(baseline, retrieval, None, 
                                                                                      baseline.train_data, 
                                                                                      baseline.dup_sets_train,
                                                                                      bug_train_ids,
                                                                                      batch_size_test, 1,
                                                                                      issues_by_buckets)

validation_sample = [valid_input_sample['title']['token'], valid_input_sample['title']['segment'], 
                   valid_input_sample['description']['token'], valid_input_sample['description']['segment'],
                   valid_input_sample['info'], valid_sim]

# Categorical columns
number_of_columns_info = valid_input_sample['info'].shape[1]
# Max sequence title
MAX_SEQUENCE_LENGTH_T = valid_input_sample['title']['token'].shape[1]
MAX_SEQUENCE_LENGTH_D = valid_input_sample['description']['token'].shape[1]

In [23]:
valid_input_sample['title']['token'].shape, \
valid_input_sample['description']['token'].shape, \
valid_input_sample['title']['segment'].shape, \
valid_input_sample['description']['segment'].shape, \
valid_input_sample['info'].shape, valid_sim.shape

((384, 20), (384, 20), (384, 20), (384, 20), (384, 729), (384,))

### Validar entrada

In [24]:
# %%time 

#baseline.display_batch(baseline.train_data, baseline.dup_sets_train, bug_train_ids, 5)

In [25]:
"Test ", len(baseline.test_data)

('Test ', 7986)

## Pre-trained embeddings

Loading pretrained word vectors

### Glove

In [26]:
vocab = baseline.load_vocabulary(os.path.join(DIR, 'vocab_embed.pkl'))
#print(np.random.choice(vocab, 10))
# for token in vocab:
#     print(token)

vocabulary loaded


In [27]:
"Total vocabulary: {}".format(len(vocab))

'Total vocabulary: 18562'

In [28]:
def generating_embed(baseline, GLOVE_DIR, EMBEDDING_DIM):
    embeddings_index = {}
    embed_path = os.path.join(GLOVE_DIR, 'glove.42B.300d.txt')
    f = open(embed_path, 'rb')
    #num_lines = sum(1 for line in open(embed_path, 'rb'))

    vocab = baseline.load_vocabulary(os.path.join(baseline.DIR, 'vocab_embed.pkl'))
    vocab_size = len(vocab) 

    # Initialize uniform the vector considering the Tanh activation
    embedding_matrix = np.random.uniform(-1.0, 1.0, (vocab_size, EMBEDDING_DIM))
    embedding_matrix[0, :] = np.zeros(EMBEDDING_DIM)

    loop = tqdm(f)
    loop.set_description("Loading Glove")
    for line in loop:
        tokens = line.split()
        word = tokens[0]
        embeddings_index[word] = np.asarray(tokens[1:], dtype='float32')
        loop.update(1)
    f.close()
    loop.close()

    print('Total %s word vectors in Glove 42B 300d.' % len(embeddings_index))

    loop = tqdm(total=vocab_size)
    loop.set_description('Loading embedding from dataset pretrained')
    i = 0
    for word, embed in vocab.items():
        if word in embeddings_index:
            embedding_matrix[i] = embeddings_index[word]
        else:
            embedding_matrix[i] = np.asarray(embed, dtype='float32')
        loop.update(1)
        i+=1
    loop.close()
    baseline.embedding_matrix = embedding_matrix

In [29]:
%%time

generating_embed(baseline, GLOVE_DIR=GLOVE_DIR, EMBEDDING_DIM=EMBEDDING_DIM) # MAX_NB_WORDS=MAX_NB_WORDS

vocabulary loaded



Total 1917494 word vectors in Glove 42B 300d.



CPU times: user 1min 20s, sys: 2.69 s, total: 1min 23s
Wall time: 1min 22s


## Experiment

## Towards Accurate Duplicate Bug Retrieval Using Deep Learning Techniques

https://github.com/tqtg/DuplicateBugFinder

### Embedding layer

In [30]:
from keras.constraints import MaxNorm
from keras.initializers import TruncatedNormal, RandomUniform

# Is missing the padding_idx used in pytorch
# https://pytorch.org/docs/stable/_modules/torch/nn/modules/sparse.html
# https://stackoverflow.com/questions/54824768/rnn-model-gru-of-word2vec-to-regression-not-learning
def embedding_layer(embeddings, num_words, embedding_dim, max_sequence_length, trainable):
    embedding_layer = Embedding(num_words,
                                  embedding_dim,
                                  name='embedding_layer',
                                  weights=[embeddings],
                                  embeddings_constraint=MaxNorm(max_value=1, axis=0),
                                  #input_length=max_sequence_length,
                                  input_length=None,
                                  trainable=trainable)
    return embedding_layer

### BERT

In [31]:
from keras_bert import load_trained_model_from_checkpoint
from keras_bert import compile_model, get_model
from keras.layers import GlobalAveragePooling1D

def bert_model(MAX_SEQUENCE_LENGTH, name):
    layer_num = 8
#     model = load_trained_model_from_checkpoint(
#             config_path,
#             model_path,
#             training=True,
#             trainable=True,
#             seq_len=MAX_SEQUENCE_LENGTH,
#     )
    model = load_trained_model_from_checkpoint(
        config_path,
        model_path,
        training=True,
        use_adapter=True,
        seq_len=MAX_SEQUENCE_LENGTH,
        trainable=['Encoder-{}-MultiHeadSelfAttention-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Adapter'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-MultiHeadSelfAttention-Norm'.format(i + 1) for i in range(12-layer_num, 13)] +
        ['Encoder-{}-FeedForward-Norm'.format(i + 1) for i in range(layer_num)],
    )
#     model = get_model(
#         token_num=len(token_dict),
#         head_num=10,
#         transformer_num=layer_num,
#         embed_dim=100,
#         feed_forward_dim=100,
#         seq_len=MAX_SEQUENCE_LENGTH,
#         pos_num=MAX_SEQUENCE_LENGTH,
#         dropout_rate=0.05,
#     )
    compile_model(model)
    inputs = model.inputs[:2]
    outputs = model.get_layer('Encoder-{}-FeedForward-Norm'.format(layer_num)).output
    #outputs = model.get_layer('Extract').output
    outputs = GlobalAveragePooling1D()(outputs)
    outputs = Dense(300, activation='tanh')(outputs)
    
    model = Model(inputs, outputs, name='FeatureBERTGenerationModel{}'.format(name))
    
    return model

### MLP

In [32]:
def mlp_model(input_size):
    info_input = Input(shape=(input_size, ), name='Feature_BugInput')
    input_size = 300
    
    for units in [64, 32]:
        layer = Dense(units, activation='tanh', kernel_initializer='random_uniform')(info_input)
    
    layer = Dense(input_size, activation='tanh')(info_input)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureMlpGenerationModel')
    
    return mlp_feature_model

### Siamese model

In [33]:
def pairwise_distance(feature, squared=False):
    """Computes the pairwise distance matrix with numerical stability.

    output[i, j] = || feature[i, :] - feature[j, :] ||_2

    Args:
      feature: 2-D Tensor of size [number of data, feature dimension].
      squared: Boolean, whether or not to square the pairwise distances.

    Returns:
      pairwise_distances: 2-D Tensor of size [number of data, number of data].
    """
    pairwise_distances_squared = math_ops.add(
        math_ops.reduce_sum(math_ops.square(feature), axis=[1], keepdims=True),
        math_ops.reduce_sum(
            math_ops.square(array_ops.transpose(feature)),
            axis=[0],
            keepdims=True)) - 2.0 * math_ops.matmul(feature,
                                                    array_ops.transpose(feature))

    # Deal with numerical inaccuracies. Set small negatives to zero.
    pairwise_distances_squared = math_ops.maximum(pairwise_distances_squared, 0.0)
    # Get the mask where the zero distances are at.
    error_mask = math_ops.less_equal(pairwise_distances_squared, 0.0)

    # Optionally take the sqrt.
    if squared:
        pairwise_distances = pairwise_distances_squared
    else:
        pairwise_distances = math_ops.sqrt(
            pairwise_distances_squared + math_ops.to_float(error_mask) * 1e-16)

    # Undo conditionally adding 1e-16.
    pairwise_distances = math_ops.multiply(
        pairwise_distances, math_ops.to_float(math_ops.logical_not(error_mask)))

    num_data = array_ops.shape(feature)[0]
    # Explicitly set diagonals to zero.
    mask_offdiagonals = array_ops.ones_like(pairwise_distances) - array_ops.diag(
        array_ops.ones([num_data]))
    pairwise_distances = math_ops.multiply(pairwise_distances, mask_offdiagonals)
    return pairwise_distances

def masked_maximum(data, mask, dim=1):
    """Computes the axis wise maximum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the maximum.

    Returns:
      masked_maximums: N-D `Tensor`.
        The maximized dimension is of size 1 after the operation.
    """
    axis_minimums = math_ops.reduce_min(data, dim, keepdims=True)
    masked_maximums = math_ops.reduce_max(
        math_ops.multiply(data - axis_minimums, mask), dim,
        keepdims=True) + axis_minimums
    return masked_maximums

def masked_minimum(data, mask, dim=1):
    """Computes the axis wise minimum over chosen elements.

    Args:
      data: 2-D float `Tensor` of size [n, m].
      mask: 2-D Boolean `Tensor` of size [n, m].
      dim: The dimension over which to compute the minimum.

    Returns:
      masked_minimums: N-D `Tensor`.
        The minimized dimension is of size 1 after the operation.
    """
    axis_maximums = math_ops.reduce_max(data, dim, keepdims=True)
    masked_minimums = math_ops.reduce_min(
        math_ops.multiply(data - axis_maximums, mask), dim,
        keepdims=True) + axis_maximums
    return masked_minimums

In [34]:
## required for semi-hard triplet loss:
from tensorflow.python.ops import array_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.framework import dtypes
import tensorflow as tf

def quintet_loss(y_true, y_pred):
    return triplet_loss(y_pred)

def triplet_loss(vects):
    #del y_true
    margin = 1.
    labels = vects[:, :1]
 
    labels = tf.cast(labels, dtype='int32')

    embeddings = vects[:, 1:]

    ### Code from Tensorflow function [tf.contrib.losses.metric_learning.triplet_semihard_loss] starts here:
    
    # Reshape [batch_size] label tensor to a [batch_size, 1] label tensor.
    # lshape=array_ops.shape(labels)
    # assert lshape.shape == 1
    # labels = array_ops.reshape(labels, [lshape[0], 1])

    # Build pairwise squared distance matrix.
    pdist_matrix = pairwise_distance(embeddings, squared=True)
    # Build pairwise binary adjacency matrix.
    adjacency = math_ops.equal(labels, array_ops.transpose(labels))
    # Invert so we can select negatives only.
    adjacency_not = math_ops.logical_not(adjacency)

    # global batch_size  
    batch_size = array_ops.size(labels) # was 'array_ops.size(labels)'

    # Compute the mask.
    pdist_matrix_tile = array_ops.tile(pdist_matrix, [batch_size, 1])
    mask = math_ops.logical_and(
        array_ops.tile(adjacency_not, [batch_size, 1]),
        math_ops.greater(
            pdist_matrix_tile, array_ops.reshape(
                array_ops.transpose(pdist_matrix), [-1, 1])))
    mask_final = array_ops.reshape(
        math_ops.greater(
            math_ops.reduce_sum(
                math_ops.cast(mask, dtype=dtypes.float32), 1, keepdims=True),
            0.0), [batch_size, batch_size])
    mask_final = array_ops.transpose(mask_final)

    adjacency_not = math_ops.cast(adjacency_not, dtype=dtypes.float32)
    mask = math_ops.cast(mask, dtype=dtypes.float32)

    # negatives_outside: smallest D_an where D_an > D_ap.
    negatives_outside = array_ops.reshape(
        masked_minimum(pdist_matrix_tile, mask), [batch_size, batch_size])
    negatives_outside = array_ops.transpose(negatives_outside)

    # negatives_inside: largest D_an.
    negatives_inside = array_ops.tile(
        masked_maximum(pdist_matrix, adjacency_not), [1, batch_size])
    semi_hard_negatives = array_ops.where(
        mask_final, negatives_outside, negatives_inside)

    loss_mat = math_ops.add(margin, pdist_matrix - semi_hard_negatives)

    mask_positives = math_ops.cast(
        adjacency, dtype=dtypes.float32) - array_ops.diag(
        array_ops.ones([batch_size]))

    # In lifted-struct, the authors multiply 0.5 for upper triangular
    #   in semihard, they take all positive pairs except the diagonal.
    num_positives = math_ops.reduce_sum(mask_positives)

    semi_hard_triplet_loss_distance = math_ops.truediv(
        math_ops.reduce_sum(
            math_ops.maximum(
                math_ops.multiply(loss_mat, mask_positives), 0.0)),
        num_positives,
        name='triplet_semihard_loss')
    
    ### Code from Tensorflow function semi-hard triplet loss ENDS here.
    return semi_hard_triplet_loss_distance

In [35]:
from keras.layers import concatenate, Add, Lambda, merge, Average, Maximum
from keras.optimizers import Adam, Nadam

def siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, sequence_length_info, 
                  sequence_length_t, sequence_length_d, name):
  
    # Title
    bug_t_token = Input(shape = (sequence_length_t, ), name = 'title_token_{}'.format(name))
    bug_t_segment = Input(shape = (sequence_length_t, ), name = 'title_segment_{}'.format(name))
    # Description
    bug_d_token = Input(shape = (sequence_length_d, ), name = 'desc_token_{}'.format(name))
    bug_d_segment = Input(shape = (sequence_length_d, ), name = 'desc_segment_{}'.format(name))
    # Categorical
    bug_i = Input(shape = (sequence_length_info, ), name = 'info_{}'.format(name))
    
    bug_t_feat = title_feature_model([bug_t_token, bug_t_segment])
    bug_d_feat = desc_feature_model([bug_d_token, bug_d_segment])
    bug_i_feat = categorical_feature_model(bug_i)
    
    #bug_feature_output = Add(name = 'merge_features_{}'.format(name))([bug_i_feat, bug_t_feat, bug_d_feat])
    bug_feature_output = concatenate([bug_i_feat, bug_t_feat, bug_d_feat], name = 'merge_features_{}'.format(name))
    
    bug_feature_model = Model(inputs=[bug_t_token, bug_t_segment, bug_d_token, bug_d_segment, bug_i], outputs=[bug_feature_output], name = 'merge_features_{}'.format(name))
    
    return bug_feature_model

In [36]:
def max_margin_objective(encoded_anchor, decay_lr=1):
    
    input_labels = Input(shape=(1,), name='input_label')    # input layer for labels
    inputs = np.concatenate([encoded_anchor.input, [input_labels]], -1).tolist()
    
    encoded_anchor = encoded_anchor.output
    
    output = concatenate([input_labels, encoded_anchor])  # concatenating the labels + embeddings
    
    similarity_model = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

    #optimizer = Nadam(lr=1e-3, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.01)
    # optimizer = Adam(lr=1e-3 * decay_lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
    
    # setup the optimization process 
    similarity_model.compile(optimizer='adam', loss=quintet_loss) 
    # metrics=[pos_distance, neg_distance, custom_margin_loss]

    return similarity_model

In [ ]:
import _pickle as pickle
def save_loss(result):
    with open(os.path.join(DIR,'{}_log.pkl'.format(METHOD)), 'wb') as f:
        pickle.dump(result, f)
    print("=> result saved!")

In [ ]:
# Domain to use
METHOD = 'deepTL_{}'.format(limit_train)
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

In [37]:
# %%time
import keras

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

# Clear GPU memory
# from numba import cuda
# cuda.select_device(0)
# cuda.close()

# Feature models
'''
    cnn_model
    lstm_model
    mlp_model
'''
title_feature_model = bert_model(MAX_SEQUENCE_LENGTH_T, 'Title')
desc_feature_model = bert_model(MAX_SEQUENCE_LENGTH_D, 'Description')
categorical_feature_model = mlp_model(number_of_columns_info)

# Similarity model
encoded_anchor = siamese_model(title_feature_model, desc_feature_model, categorical_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D, 'in')

similarity_model = max_margin_objective(encoded_anchor, decay_lr=1)

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()

'''
    Experiment
'''
result = { 'train' : [], 'test' : [] }
limit_train = int(epochs * freeze_train) # 10% de 1000 , 100 epocas
print("Total of ", limit_train)
for epoch in range(limit_train):
    batch_triplet_train, \
        train_input_sample, train_sim = batch_iterator(baseline, retrieval, encoded_anchor, baseline.train_data, 
                                                       baseline.dup_sets_train, bug_train_ids, 
                                                           batch_size, 1, issues_by_buckets, TRIPLET_HARD=False)
    train_batch = [train_input_sample['title']['token'], train_input_sample['title']['segment'], 
                   train_input_sample['description']['token'], train_input_sample['description']['segment'],
                   train_input_sample['info'], train_sim]
    
#     if epoch == 10:
#         similarity_model = max_margin_objective(encoded_anchor, encoded_positive, encoded_negative, decay_lr=0.1)
    
    h = similarity_model.train_on_batch(x=train_batch, y=train_sim)
    h_validation = similarity_model.test_on_batch(x=validation_sample, y=valid_sim)
    
    # save results
    result['train'].append([h])
    result['test'].append([h_validation])
    
    if( (epoch+1) % 10 == 0 or (epoch+1 == epochs) ):
        save_loss(result)
    
    if (epoch+1 == epochs): #(epoch > 1 and epoch % 10 == 0) or (epoch+1 == epochs):
        recall, exported_rank, debug = experiment.evaluate_validation_test(retrieval, verbose, encoded_anchor, issues_by_buckets, 
                                                               bug_train_ids, method='bert')
        print("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}, recall@25: {:.2f}".format(epoch+1, h, h_validation, recall))
    else:
        print("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}".format(epoch+1, h, h_validation))
    loss = h
    
    if loss < best_loss:
        best_loss = loss
        best_epoch = epoch+1

#experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(epochs)))
#experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs)), verbose=1)
print('Best_epoch={}, Best_loss={:.2f}, Recall@25={:.2f}'.format(best_epoch, best_loss, recall))

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 729)          0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 20)           0                                            
__________________________________________________________________________________________________
desc_token_in (InputLayer)      (None, 20)           0                                            
__________________________________________________________________________________________________
desc_segme

Epoch: 235 Loss: 0.02
Epoch: 236 Loss: 0.03
Epoch: 237 Loss: 0.02
Epoch: 238 Loss: 0.02
Epoch: 239 Loss: 0.03
Epoch: 240 Loss: 0.02
Epoch: 241 Loss: 0.02
Epoch: 242 Loss: 0.02
Epoch: 243 Loss: 0.02
Epoch: 244 Loss: 0.01
Epoch: 245 Loss: 0.03
Epoch: 246 Loss: 0.01
Epoch: 247 Loss: 0.03
Epoch: 248 Loss: 0.02
Epoch: 249 Loss: 0.02
Epoch: 250 Loss: 0.01
Epoch: 251 Loss: 0.04
Epoch: 252 Loss: 0.01
Epoch: 253 Loss: 0.04
Epoch: 254 Loss: 0.01
Epoch: 255 Loss: 0.02
Epoch: 256 Loss: 0.02
Epoch: 257 Loss: 0.02
Epoch: 258 Loss: 0.01
Epoch: 259 Loss: 0.02
Epoch: 260 Loss: 0.02
Epoch: 261 Loss: 0.04
Epoch: 262 Loss: 0.02
Epoch: 263 Loss: 0.02
Epoch: 264 Loss: 0.02
Epoch: 265 Loss: 0.03
Epoch: 266 Loss: 0.02
Epoch: 267 Loss: 0.02
Epoch: 268 Loss: 0.02
Epoch: 269 Loss: 0.05
Epoch: 270 Loss: 0.03
Epoch: 271 Loss: 0.01
Epoch: 272 Loss: 0.01
Epoch: 273 Loss: 0.02
Epoch: 274 Loss: 0.01
Epoch: 275 Loss: 0.02
Epoch: 276 Loss: 0.02
Epoch: 277 Loss: 0.03
Epoch: 278 Loss: 0.06
Epoch: 279 Loss: 0.01
Epoch: 280

Epoch: 608 Loss: 0.01
Epoch: 609 Loss: 0.01
Epoch: 610 Loss: 0.00
Epoch: 611 Loss: 0.02
Epoch: 612 Loss: 0.01
Epoch: 613 Loss: 0.01
Epoch: 614 Loss: 0.03
Epoch: 615 Loss: 0.02
Epoch: 616 Loss: 0.01
Epoch: 617 Loss: 0.01
Epoch: 618 Loss: 0.01
Epoch: 619 Loss: 0.01
Epoch: 620 Loss: 0.02
Epoch: 621 Loss: 0.01
Epoch: 622 Loss: 0.01
Epoch: 623 Loss: 0.00
Epoch: 624 Loss: 0.01
Epoch: 625 Loss: 0.01
Epoch: 626 Loss: 0.00
Epoch: 627 Loss: 0.01
Epoch: 628 Loss: 0.02
Epoch: 629 Loss: 0.01
Epoch: 630 Loss: 0.01
Epoch: 631 Loss: 0.02
Epoch: 632 Loss: 0.02
Epoch: 633 Loss: 0.00
Epoch: 634 Loss: 0.01
Epoch: 635 Loss: 0.01
Epoch: 636 Loss: 0.02
Epoch: 637 Loss: 0.01
Epoch: 638 Loss: 0.01
Epoch: 639 Loss: 0.01
Epoch: 640 Loss: 0.01
Epoch: 641 Loss: 0.01
Epoch: 642 Loss: 0.01
Epoch: 643 Loss: 0.01
Epoch: 644 Loss: 0.01
Epoch: 645 Loss: 0.01
Epoch: 646 Loss: 0.01
Epoch: 647 Loss: 0.01
Epoch: 648 Loss: 0.01
Epoch: 649 Loss: 0.01
Epoch: 650 Loss: 0.01
Epoch: 651 Loss: 0.01
Epoch: 652 Loss: 0.01
Epoch: 653

Epoch: 981 Loss: 0.01
Epoch: 982 Loss: 0.02
Epoch: 983 Loss: 0.01
Epoch: 984 Loss: 0.01
Epoch: 985 Loss: 0.01
Epoch: 986 Loss: 0.01
Epoch: 987 Loss: 0.01
Epoch: 988 Loss: 0.01
Epoch: 989 Loss: 0.01
Epoch: 990 Loss: 0.02
Epoch: 991 Loss: 0.01
Epoch: 992 Loss: 0.02
Epoch: 993 Loss: 0.01
Epoch: 994 Loss: 0.01
Epoch: 995 Loss: 0.02
Epoch: 996 Loss: 0.01
Epoch: 997 Loss: 0.01
Epoch: 998 Loss: 0.01
Epoch: 999 Loss: 0.00
Epoch: 1000 Loss: 0.01, recall@25: 0.69
Saved model 'modelos/model_bert_preprocessing_deepTL_1000_feature_1000epochs_64batch(openoffice).h5' to disk
Best_epoch=903, Best_loss=0.00, Recall@25=0.69


In [ ]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

In [ ]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [ ]:
experiment.save_model(similarity_model, SAVE_PATH.replace('@number_of_epochs@', str(limit_train)))
experiment.save_model(encoded_anchor, SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(limit_train)), verbose=1)

In [ ]:
len(result['train']), len(result['test'])

In [ ]:
model = similarity_model.get_layer('concatenate_1')
output = model.output
inputs = similarity_model.inputs
model = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

# setup the optimization process 
model.compile(optimizer='adam', loss=quintet_loss)
model.summary()

In [ ]:
# Domain to use
METHOD = 'deepTL_{}'.format(epochs)
SAVE_PATH = '{}_preprocessing_{}_feature@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)
SAVE_PATH_FEATURE = '{}_preprocessing_{}_feature_@number_of_epochs@epochs_64batch({})'.format(PREPROCESSING, METHOD, DOMAIN)

In [ ]:
end_train = epochs - limit_train
for epoch in range(limit_train, end_train):
    batch_triplet_train, \
        train_input_sample, train_sim = batch_iterator(baseline, retrieval, model, baseline.train_data, 
                                                       baseline.dup_sets_train, bug_train_ids, 
                                                           batch_size, 1, issues_by_buckets, TRIPLET_HARD=False)
    train_batch = [train_input_sample['title']['token'], train_input_sample['title']['segment'], 
                   train_input_sample['description']['token'], train_input_sample['description']['segment'],
                   train_input_sample['info'], train_sim]
    

    h = model.train_on_batch(x=train_batch, y=train_sim)
    h_validation = model.test_on_batch(x=validation_sample, y=valid_sim)
    
    # save results
    result['train'].append(h)
    result['test'].append(h_validation)
    
    if( (epoch+1) % 10 == 0 or (epoch+1 == end_train )):
        save_loss(result)
    
    print(("Epoch: {} Loss: {:.2f}, Loss_test: {:.2f}\n" +
               "TL_w: {:.2f}, TL_pos_w: {:.2f}, TL_neg_w: {:.2f}, TL_centroid_w: {:.2f}\n" + 
              "TL: {:.2f}, TL_pos: {:.2f}, TL_neg: {:.2f}, TL_centroid: {:.2f}").format(
            epoch+1, h[0], h_validation[0], h[1], h[2], h[3], h[4], h[5], h[6], h[7], h[8]))

In [ ]:
len(result['train']), len(result['test'])

In [ ]:
encoded = model.get_layer('merge_features_in')
output = encoded.output
inputs = similarity_model.inputs[:-1]
encoded_anchor = Model(inputs = inputs, outputs = output, name = 'Similarity_Model')

In [ ]:
SAVE_PATH.replace('@number_of_epochs@', str(epochs))

In [38]:
recall

0.69

In [39]:
exported_rank[:20]

['98306:88871,50853,33630,90791|88871:0.6935537159442902,62750:0.6791682839393616,42169:0.6724690794944763,78716:0.669046014547348,59051:0.6671613156795502,113851:0.6655094027519226,35644:0.6607243418693542,37651:0.6605731546878815,69738:0.6599004864692688,95954:0.6581278741359711,66329:0.6570902466773987,47227:0.656943291425705,33257:0.6566915214061737,54812:0.6565796136856079,99507:0.6562356650829315,92237:0.6561053693294525,1685:0.655159205198288,54357:0.6550473868846893,97116:0.6549574732780457,48861:0.6536872684955597,107843:0.6528923213481903,38090:0.6516199707984924,77976:0.6513862013816833,56216:0.6508936285972595,61627:0.6506760716438293,75024:0.6504048705101013,87639:0.6501749753952026,75171:0.6497997939586639,96878:0.6495416462421417',
 '32771:32490,33548,32560,33879,32699|5412:0.69867804646492,7354:0.6884078085422516,33506:0.6871865391731262,9917:0.6863522231578827,71665:0.6858185827732086,14831:0.685621827840805,6789:0.6847501695156097,4516:0.6821760833263397,67340:0.68171

### Retrieval evaluation

In [41]:
print("Total of queries:", len(retrieval.test))

Total of queries: 7986


#### Getting the model trained

In [42]:
SAVE_PATH_FEATURE.replace('@number_of_epochs@', str(epochs))

'bert_preprocessing_deepTL_1000_feature_1000epochs_64batch(openoffice)'

In [44]:
encoded_anchor.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 729)          0                                            
__________________________________________________________________________________________________
title_token_in (InputLayer)     (None, 20)           0                                            
__________________________________________________________________________________________________
title_segment_in (InputLayer)   (None, 20)           0                                            
__________________________________________________________________________________________________
desc_token_in (InputLayer)      (None, 20)           0                                            
__________________________________________________________________________________________________
desc_segme

In [ ]:
len(exported_rank)

In [47]:
EXPORT_RANK_PATH = os.path.join(DIR, 'exported_rank_{}.txt'.format(METHOD))
EXPORT_RANK_PATH

'data/processed/openoffice/bert/exported_rank_deepTL_1000.txt'

In [48]:
with open(EXPORT_RANK_PATH, 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [49]:
report = experiment.evaluation.evaluate(EXPORT_RANK_PATH)
report

{'1 - recall_at_5': 0.56,
 '2 - recall_at_10': 0.62,
 '3 - recall_at_15': 0.65,
 '4 - recall_at_20': 0.67,
 '5 - recall_at_25': 0.69}

#### Some ideas to visualizate

- https://towardsdatascience.com/building-a-recommendation-system-using-neural-network-embeddings-1ef92e5c80c9